In [1]:
import os
import pickle

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from src.data_preparation.data_preprocessing import DataReshaperLSTM
from src.visualization.charts import ModelCharts
from src.visualization.metrics import PredictionEvaluator
from src.config import Config

## Load the data

In [2]:
config = Config()
variant_co2 = 'co2_per_capita'
train_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/train.csv')).set_index(["year", config.additional_index])
test_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/test.csv')).set_index(["year", config.additional_index])

with open(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/data_preprocessor_lstm.pkl'), 'rb') as f:
    data_preprocessor_lstm = pickle.load(f)

In [3]:
train_df

,,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,land_use_change_co2_t-2,oil_co2_t-2,...,population_t-1,gdp_t-1,temperature_change_from_co2_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1930,Argentina,0,0.006943,0.175266,0.507331,0.016325,0.048410,0.000000,0.003933,0.311606,0.022520,...,0.007087,0.174392,0.493851,0.022711,0.047334,0.000000,0.005904,0.306015,0.022659,0.236081
1931,Argentina,0,0.007176,0.178610,0.493851,0.023836,0.047334,0.000000,0.005904,0.306015,0.022659,...,0.007303,0.162400,0.481904,0.024072,0.045068,0.000000,0.005711,0.252516,0.021200,0.199768
1932,Argentina,0,0.007394,0.166327,0.481904,0.025265,0.045068,0.000000,0.005711,0.252516,0.021200,...,0.007503,0.146788,0.471364,0.033014,0.037719,0.000000,0.006939,0.217711,0.026236,0.179355
1933,Argentina,0,0.007595,0.150338,0.471364,0.034650,0.037719,0.000000,0.006939,0.217711,0.026236,...,0.007686,0.138581,0.462130,0.030168,0.033701,0.000000,0.009276,0.198031,0.029199,0.165318
1934,Argentina,0,0.007779,0.141933,0.462130,0.031663,0.033701,0.000000,0.009276,0.198031,0.029199,...,0.007850,0.142834,0.454119,0.030486,0.031171,0.000000,0.012256,0.184591,0.029855,0.155812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003,Vietnam,44,0.059659,0.067622,0.070801,0.129113,0.020349,0.001616,0.004740,0.093405,0.019882,...,0.059873,0.070411,0.070078,0.153245,0.022190,0.001236,0.007499,0.089355,0.022043,0.068187
2004,Vietnam,44,0.060292,0.072114,0.070078,0.160839,0.022190,0.001236,0.007499,0.089355,0.022043,...,0.060508,0.075243,0.069361,0.171695,0.025545,0.001445,0.009425,0.092293,0.022477,0.070668
2005,Vietnam,44,0.060932,0.077062,0.069361,0.180204,0.025545,0.001445,0.009425,0.092293,0.022477,...,0.061146,0.080752,0.068657,0.186919,0.027515,0.001409,0.015698,0.093135,0.025559,0.064508


## Reshape the data

In [4]:
data_resherper = DataReshaperLSTM()
x_train, x_test, y_train, y_test = data_resherper.reshape_data(train_df, test_df)

## Build the model

In [5]:
input_shape = (x_train.shape[1], x_train.shape[2])
output_units = 1

model = Sequential([
    LSTM(50, input_shape=input_shape, return_sequences = True),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(output_units, activation='relu')
])

model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             14200     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 34,451
Trainable params: 34,451
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [6]:
history = model.fit(x_train, y_train, epochs=config.epochs, batch_size=config.batch_size)

Epoch 1/15
3103/3103 [==============================] - 4s 842us/step - loss: 0.0064
Epoch 2/15
3103/3103 [==============================] - 2s 757us/step - loss: 0.0029
Epoch 3/15
3103/3103 [==============================] - 2s 683us/step - loss: 0.0021
Epoch 4/15
3103/3103 [==============================] - 2s 682us/step - loss: 0.0020
Epoch 5/15
3103/3103 [==============================] - 2s 676us/step - loss: 0.0016
Epoch 6/15
3103/3103 [==============================] - 2s 676us/step - loss: 0.0018
Epoch 7/15
3103/3103 [==============================] - 2s 681us/step - loss: 0.0016
Epoch 8/15
3103/3103 [==============================] - 2s 695us/step - loss: 0.0016
Epoch 9/15
3103/3103 [==============================] - 2s 675us/step - loss: 0.0014
Epoch 10/15
3103/3103 [==============================] - 2s 675us/step - loss: 0.0014
Epoch 11/15
3103/3103 [==============================] - 2s 677us/step - loss: 0.0014
Epoch 12/15
3103/3103 [==============================] - 2s 675

In [7]:
# Save the model
model.save(os.path.join(config.models_folder, f'{variant_co2}_lstm_model.h5'))

# Evaluate the model on the test set
loss = model.evaluate(x_test, y_test, verbose = 0)

## Predictions

In [8]:
train_predictions = model.predict(x_train)
test_predictions = model.predict(x_test)

inverted_data_predicted_train_y = data_preprocessor_lstm.inverse_transform_data(train_predictions, train_predictions.shape[0], train_df.shape[1]-2)
inverted_data_train_y = data_preprocessor_lstm.inverse_transform_data(y_train, train_predictions.shape[0], train_df.shape[1]-2)

inverted_data_predicted_test_y = data_preprocessor_lstm.inverse_transform_data(test_predictions, test_predictions.shape[0], test_df.shape[1]-2)
inverted_data_test_y = data_preprocessor_lstm.inverse_transform_data(y_test, test_predictions.shape[0], test_df.shape[1]-2)

26/26 [==============================] - 0s 418us/step


In [9]:
train_df_reset = train_df.reset_index()
test_df_reset = test_df.reset_index()

co2_predicted_train = inverted_data_predicted_train_y[:, -1]
co2_actual_train = inverted_data_train_y[:, -1]
co2_predicted_test = inverted_data_predicted_test_y[:, -1]
co2_actual_test = inverted_data_test_y[:, -1]

# Create DataFrames for train and test
train_results = pd.DataFrame({
    "country": train_df_reset["country_index"].values,
    "year": train_df_reset["year"].values,
    "co2_predicted": co2_predicted_train,
    "co2_actual": co2_actual_train
})

test_results = pd.DataFrame({
    "country": test_df_reset["country_index"].values,
    "year": test_df_reset["year"].values,
    "co2_predicted": co2_predicted_test,
    "co2_actual": co2_actual_test
})


train_results.to_csv(os.path.join(config.predictions_lstm, f'{variant_co2}_train.csv'))
test_results.to_csv(os.path.join(config.predictions_lstm, f'{variant_co2}_test.csv'))

## Charts

In [10]:
charts = ModelCharts(train_results, test_results)

#Line plot
charts.generate_line_plot(variant = variant_co2, model_output_file=config.predictions_lstm)
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='train')
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='test')

#Scatter plot
charts.generate_scatter_plot(variant = variant_co2, model_output_file=config.predictions_lstm)
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='train')
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='test')

## Metrics

In [11]:
evaluator = PredictionEvaluator()
evaluator.evaluate(train_results, test_results, actual_col='co2_actual', predicted_col='co2_predicted', variant = variant_co2, model_output_file=config.metrics_lstm)